In [1]:
import numpy as np
from qutip import *
from helpers.estimation import *
from helpers.data_processing import *
from helpers.targets import *
import exp_params
import scipy as sc

USING ORENS OBSERVABLES


### Check yourself: print function, dimension, and number of displacements

In [2]:
print(f"{exp_params.which_function}\nD={exp_params.D}\nnD={exp_params.nD}")

Q
D=6
nD=35


### State list 

In [3]:
state_list = exp_params.cat_state_list
print(state_list)

['cat-eve-1' 'cat-nmp-1' 'cat-nop-1' 'cat-odd-1']


### Choose whether to use grape or ideal for state preparation and target states, and specify target state directory

In [4]:
use_grape = True

# GRAPE generated target states
target_states_directory = r"C:\Users\clara\OneDrive - FontaineShim\Documents\Singapore\scripts\tomography\target_states"

### Gather linear inversion variables used to generate Least-Squares estimator, $\rho_{LS}$

In [5]:
# Load inverse map variables for converting pe to rho
map_variables = np.load(
    f"map_variables\map_variables_D={exp_params.D}_nD={exp_params.nD}_Q.npz"
)
W = map_variables["W"]
beta = map_variables["beta"]


### Function for protocol simulation with QuTiP

In [6]:
def pns_sim(rho_init, frq, c_ops_qc):
    H0 = (
        -2 * np.pi * frq * exp_params.Qd * exp_params.Q
        - 2 * np.pi
        * exp_params.chi
        * exp_params.Qd
        * exp_params.Q
        * exp_params.Cd
        * exp_params.C
        # - 2
        # * np.pi
        # * exp_params.chi_prime
        # / 2
        # * exp_params.Qd
        # * exp_params.Q
        # * exp_params.Cd
        # * exp_params.Cd
        # * exp_params.C
        # * exp_params.C
        - 2
        * np.pi
        * exp_params.Kerr
        / 2
        * exp_params.Cd
        * exp_params.Cd
        * exp_params.C
        * exp_params.C
        - 2
        * np.pi
        * exp_params.alpha
        / 2
        * exp_params.Qd
        * exp_params.Qd
        * exp_params.Q
        * exp_params.Q
    )
    Hd = 2 * np.pi * exp_params.power_rabi_A_Q * 1j * (exp_params.Qd - exp_params.Q)

    tlist_pi_selective = np.linspace(0, exp_params.sigma_Q * exp_params.chop_Q, 101)
    H = [H0, [Hd, exp_params.pulse_Q]]
    # e_ops = [
    #     tensor(ue * ue.dag(), qeye(cdim)),  # Qd * Q,
    # ]

    # options = Options(max_step = 4, nsteps = 1e6)
    results = mesolve(
        H,
        rho_init,
        tlist_pi_selective,
        c_ops=c_ops_qc,  # e_ops=e_ops
    )  # , options = options)#, progress_bar = True)

    rho_qt = results.states[-1].ptrace([0])
    return np.around((rho_qt * (exp_params.ue * exp_params.ue.dag())).tr(), 6)

### Top-level iteration through the T2 values. For each T2 value, iterate through the states, run the protocol, and record in numpy arrays the fidelities, standard deviations, and $\rho_{BME}$ for each. This saves both the normal Wigner and corrected Wigner information. One npz file per T2.

In [7]:
F_MLE = np.zeros(len(state_list), dtype=float)
F_Bayes_BME = np.zeros(len(state_list), dtype=float)

In [9]:
for k, badT2 in enumerate(exp_params.badt2_list):
    print(f"badT2={badT2}")
    for j, state_name in enumerate(state_list):
        if use_grape:
            rho_tar_qc = Y_target(
                state_name, target_states_directory, exp_params.qdim, exp_params.cdim
            )

        else:
            rho_tar_qc = tensor(
                fock_dm(exp_params.qdim, 0),
                ket2dm(cavity_target_state(state_name, exp_params.cdim)),
            )


        Pid = np.zeros(len(exp_params.disp_points))


        if use_grape:
            if state_name in exp_params.pes_after_grape_sim[:, 0]:
                pge = exp_params.pes_after_grape_sim[
                    (exp_params.pes_after_grape_sim[:, 0] == state_name), 1
                ].astype(float)

            else:
                pge = exp_params.pe_after_grape_sim_avg

        else:
            pge = 0


        w = 0

        Tphi_badt2 = 1 / (1 / badT2 - 0.5 / exp_params.T1)

        c_ops_qc = [
            # Qubit Relaxation
            np.sqrt((1 + exp_params.nbar_qb) / exp_params.T1) * exp_params.Q,
            # Qubit Thermal Excitations
            np.sqrt(exp_params.nbar_qb / exp_params.T1) * exp_params.Qd,
            # Qubit Dephasing, changed
            np.sqrt(2 / Tphi_badt2) * exp_params.Qd * exp_params.Q,
            # Cavity Relaxation
            np.sqrt((1 + exp_params.nbar_cav) / exp_params.cavT1) * exp_params.C,
            # Cavity Thermal Excitations
            np.sqrt(exp_params.nbar_cav / exp_params.cavT1) * exp_params.Cd,
        ]


        for v1 in np.arange(exp_params.TM1):
            RHO_tar = (
                tensor(

                    qeye(exp_params.qdim),
                    displace(exp_params.cdim, exp_params.disp_points[w]),
                )

                * rho_tar_qc
                * tensor(
                    qeye(exp_params.qdim),
                    displace(exp_params.cdim, -exp_params.disp_points[w]),
                )
            )


            Pid[w] = pns_sim(RHO_tar, exp_params.freq_detune_Q[1], c_ops_qc)


            # Pidid[w] = RHO_tar.ptrace(1)[1,1].real


            w += 1


        for v2 in np.arange(exp_params.TM2):
            RHO_tar = (
                tensor(

                    qeye(exp_params.qdim),
                    displace(exp_params.cdim, exp_params.disp_points[w]),
                )

                * rho_tar_qc
                * tensor(
                    qeye(exp_params.qdim),
                    displace(exp_params.cdim, -exp_params.disp_points[w]),
                )
            )


            Pid[w] = pns_sim(RHO_tar, exp_params.freq_detune_Q[2], c_ops_qc)


            # Pidid[w] = RHO_tar.ptrace(1)[2,2].real


            w += 1


        for v3 in np.arange(exp_params.TM3):
            RHO_tar = (
                tensor(

                    qeye(exp_params.qdim),
                    displace(exp_params.cdim, exp_params.disp_points[w]),
                )

                * rho_tar_qc
                * tensor(
                    qeye(exp_params.qdim),
                    displace(exp_params.cdim, -exp_params.disp_points[w]),
                )
            )


            Pid[w] = pns_sim(RHO_tar, exp_params.freq_detune_Q[3], c_ops_qc)


            # Pidid[w] = RHO_tar.ptrace(1)[3,3].real


            w += 1


        for v4 in np.arange(exp_params.TM4):
            RHO_tar = (
                tensor(

                    qeye(exp_params.qdim),
                    displace(exp_params.cdim, exp_params.disp_points[w]),
                )

                * rho_tar_qc
                * tensor(
                    qeye(exp_params.qdim),
                    displace(exp_params.cdim, -exp_params.disp_points[w]),
                )
            )


            Pid[w] = pns_sim(RHO_tar, exp_params.freq_detune_Q[4], c_ops_qc)


            # Pidid[w] = RHO_tar.ptrace(1)[4,4].real


            w += 1


        for v5 in np.arange(exp_params.TM5):
            RHO_tar = (
                tensor(

                    qeye(exp_params.qdim),
                    displace(exp_params.cdim, exp_params.disp_points[w]),
                )

                * rho_tar_qc
                * tensor(
                    qeye(exp_params.qdim),
                    displace(exp_params.cdim, -exp_params.disp_points[w]),
                )
            )


            Pid[w] = pns_sim(RHO_tar, exp_params.freq_detune_Q[5], c_ops_qc)


            # Pidid[w] = RHO_tar.ptrace(1)[5,5].real


            w += 1


        for v6 in np.arange(exp_params.TM6):
            RHO_tar = (
                tensor(

                    qeye(exp_params.qdim),
                    displace(exp_params.cdim, exp_params.disp_points[w]),
                )

                * rho_tar_qc
                * tensor(
                    qeye(exp_params.qdim),
                    displace(exp_params.cdim, -exp_params.disp_points[w]),
                )
            )


            Pid[w] = pns_sim(RHO_tar, exp_params.freq_detune_Q[6], c_ops_qc)

            # Pidid[w] = RHO_tar.ptrace(1)[5,5].real

            w += 1

        for v7 in np.arange(exp_params.TM7):
            RHO_tar = (
                tensor(

                    qeye(exp_params.qdim),
                    displace(exp_params.cdim, exp_params.disp_points[w]),
                )

                * rho_tar_qc
                * tensor(
                    qeye(exp_params.qdim),
                    displace(exp_params.cdim, -exp_params.disp_points[w]),
                )
            )


            Pid[w] = pns_sim(RHO_tar, exp_params.freq_detune_Q[7], c_ops_qc)

            # Pidid[w] = RHO_tar.ptrace(1)[5,5].real

            w += 1


        Pid = (Pid - pge) / (1 - 2 * pge)


        rho_est, qRho_est = get_LS_and_MLE_rho_est(
            Pid, W, beta, exp_params.D, exp_params.nD
        )


        rho_tar_c = Qobj(rho_tar_qc.ptrace(1)[0 : exp_params.D, 0 : exp_params.D])


        rho_tar_c = rho_tar_c / rho_tar_c.tr()  # normalise it, .unit()


        Fmean_bayes, Fstd_bayes, rho_BME = bayesian_rho_est(
            numSamp=2**10, N=exp_params.N_sim, rho_tar=rho_tar_c, rhoLS=rho_est.full()
        )


        F_Bayes_BME[j] = fidelity(rho_tar_c, Qobj(rho_BME)) ** 2


        F_MLE[j] = fidelity(rho_tar_c, qRho_est) ** 2


        print(f"state {j}: {state_name}: {F_Bayes_BME[j]}")
    if use_grape:
        np.savez(
            f"results_t2\sim\Q\grape_target_states\Bayes_F_rrno={k}_grape_sim_Q.npz",
            F_MLE=F_MLE,
            F_Bayes_BME=F_Bayes_BME,
        )

    else:
        np.savez(
            f"results_t2\sim\Q\ideal_target_states\Bayes_F_rrno={k}_ideal_sim_Q.npz",
            F_MLE=F_MLE,
            F_Bayes_BME=F_Bayes_BME,
        )

badT2=22389.12
state 0: cat-eve-1: 0.9811682414989811
